In [20]:
# Import pyspark and datetime
from pyspark.sql import SparkSession
from datetime import date, datetime, timedelta

# Build spark session
spark = SparkSession.builder.master('local[*]').appName('Analytical ETL').getOrCreate()

In [61]:
# Set current date, previous date
date_str = '2020-08-06'
curr_date = datetime.strptime(date_str, '%Y-%m-%d')
prev_date = curr_date - timedelta(days = 1)

# Read trades for current day, previous day
trade_fpath = f'EOD-load/trade/'
trade_df = spark.read.parquet(trade_fpath)
trade_df_curr = trade_df.filter(trade_df["trade_dt"] == curr_date)
trade_df_prev = trade_df.filter(trade_df["trade_dt"] == prev_date)

# Read quotes for current day
quote_fpath = f'EOD-load/quote/'
quote_df = spark.read.parquet(quote_fpath)
quote_df_curr = quote_df.filter(quote_df["trade_dt"] == curr_date)

In [62]:
# Show sample of trades and count
trade_df_curr.show(5)
trade_df_curr.count()


+--------+------+--------+--------------------+------------+-------------------+------------------+----------+
|rec_type|symbol|exchange|            event_tm|event_seq_nb|         arrival_tm|          trade_pr|  trade_dt|
+--------+------+--------+--------------------+------------+-------------------+------------------+----------+
|       T|  SYMA|    NYSE|2020-08-06 14:27:...|          40|2020-08-06 09:30:00| 77.11551864232810|2020-08-06|
|       T|  SYMB|  NASDAQ|2020-08-06 12:01:...|          20|2020-08-06 09:30:00| 32.47082290604803|2020-08-06|
|       T|  SYMA|    NYSE|2020-08-06 18:14:...|          70|2020-08-06 09:30:00| 78.23471404145394|2020-08-06|
|       T|  SYMC|  NASDAQ|2020-08-06 17:51:...|          70|2020-08-06 09:30:00|161.16013914653334|2020-08-06|
|       T|  SYMC|    NYSE|2020-08-06 16:24:...|          60|2020-08-06 09:30:00|161.56712679946747|2020-08-06|
+--------+------+--------+--------------------+------------+-------------------+------------------+----------+
o

60

In [63]:
# Show sample of trades and count
trade_df_prev.show(5)
trade_df_prev.count()

+--------+------+--------+--------------------+------------+-------------------+------------------+----------+
|rec_type|symbol|exchange|            event_tm|event_seq_nb|         arrival_tm|          trade_pr|  trade_dt|
+--------+------+--------+--------------------+------------+-------------------+------------------+----------+
|       T|  SYMA|    NYSE|2020-08-05 20:21:...|          90|2020-08-05 09:30:00| 74.60467162401571|2020-08-05|
|       T|  SYMC|    NYSE|2020-08-05 10:44:...|          10|2020-08-05 09:30:00|160.87872222739200|2020-08-05|
|       T|  SYMC|  NASDAQ|2020-08-05 20:33:...|          90|2020-08-05 09:30:00|156.85586370481641|2020-08-05|
|       T|  SYMA|    NYSE|2020-08-05 10:37:...|          10|2020-08-05 09:30:00| 79.19488165597565|2020-08-05|
|       T|  SYMC|  NASDAQ|2020-08-05 14:36:...|          40|2020-08-05 09:30:00|158.63352054377958|2020-08-05|
+--------+------+--------+--------------------+------------+-------------------+------------------+----------+
o

60

In [64]:
# Show sample of quotes and count
quote_df_curr.show(5)
quote_df_curr.count()

+--------+------+--------+--------------------+------------+-------------------+-----------------+--------+-----------------+--------+----------+
|rec_type|symbol|exchange|            event_tm|event_seq_nb|         arrival_tm|           bid_pr|bid_size|           ask_pr|ask_size|  trade_dt|
+--------+------+--------+--------------------+------------+-------------------+-----------------+--------+-----------------+--------+----------+
|       Q|  SYMA|  NASDAQ|2020-08-06 12:28:...|          24|2020-08-06 09:30:00|77.86493182539250|     100|79.70623502555300|     100|2020-08-06|
|       Q|  SYMA|  NASDAQ|2020-08-06 13:16:...|          31|2020-08-06 09:30:00|76.76463842224466|     100|78.40281762917324|     100|2020-08-06|
|       Q|  SYMB|    NYSE|2020-08-06 13:26:...|          32|2020-08-06 09:30:00|34.63300051365432|     100|36.43463023060998|     100|2020-08-06|
|       Q|  SYMA|  NASDAQ|2020-08-06 15:53:...|          52|2020-08-06 09:30:00|75.00893558576169|     100|76.78429801840223

540

In [65]:
# Create temp views for Spark SQL queries
trade_df_curr.createOrReplaceTempView("trades_curr")
trade_df_prev.createOrReplaceTempView("trades_prev")
quote_df_curr.createOrReplaceTempView("quotes_curr")

In [73]:
# Calculate trailing 30 minute moving avg trade price and update trade_df
trade_df_curr_mv_avg = spark.sql("""
    SELECT
        a.symbol
        , a.exchange
        , a.event_tm
        , a.event_seq_nb
        , a.trade_pr
        , AVG(b.trade_pr) as mov_avg_trade_pr
    FROM trades_curr a
    LEFT JOIN trades_curr b
        ON a.symbol = b.symbol
        AND a.exchange = b.exchange
        AND b.event_tm BETWEEN (a.event_tm - INTERVAL 30 MINUTES) AND a.event_tm
    GROUP BY 1,2,3,4,5
    ORDER BY event_tm ASC
""")
trade_df_curr_mv_avg.createOrReplaceTempView("trades_curr_mv_avg")
trade_df_curr.show(10)

+------+--------+--------------------+------------+------------------+--------------------+
|symbol|exchange|            event_tm|event_seq_nb|          trade_pr|    mov_avg_trade_pr|
+------+--------+--------------------+------------+------------------+--------------------+
|  SYMA|  NASDAQ|2020-08-06 10:42:...|          10| 78.93245610745132|78.93245610745132...|
|  SYMC|    NYSE|2020-08-06 10:42:...|          10|157.26690601777602|157.2669060177760...|
|  SYMC|  NASDAQ|2020-08-06 10:44:...|          10|158.81964792146815|158.8196479214681...|
|  SYMB|    NYSE|2020-08-06 10:45:...|          10| 33.85934124362505|33.85934124362505...|
|  SYMB|  NASDAQ|2020-08-06 10:47:...|          10| 32.27114286138679|32.27114286138679...|
|  SYMA|    NYSE|2020-08-06 10:49:...|          10| 74.49377354690710|74.49377354690710...|
|  SYMC|    NYSE|2020-08-06 11:52:...|          20|160.15278938513183|160.1527893851318...|
|  SYMC|  NASDAQ|2020-08-06 11:54:...|          20|160.77865685239459|160.778656

In [78]:
close_pr_df = spark.sql("""
    WITH ranked_cte as (
        SELECT 
            symbol
            , exchange
            , trade_pr
            , ROW_NUMBER() OVER (PARTITION BY symbol, exchange ORDER BY event_tm DESC) as rn
        FROM trades
    )
    SELECT
        symbol
        , exchange
        , trade_pr as close_pr
    FROM ranked_cte
    WHERE rn = 1
""")
close_pr_df.createOrReplaceTempView("close_pr")
close_pr_df.show(10)

+------+--------+------------------+
|symbol|exchange|          close_pr|
+------+--------+------------------+
|  SYMC|    NYSE|159.38221783349758|
|  SYMB|    NYSE| 35.92474523433230|
|  SYMC|  NASDAQ|156.83673366195976|
|  SYMA|  NASDAQ| 77.43719185388957|
|  SYMA|    NYSE| 76.30847130527823|
|  SYMB|  NASDAQ| 35.74626943912378|
+------+--------+------------------+



In [81]:
joined_df = spark.sql("""
        SELECT
            q.rec_type
            , q.trade_dt
            , q.symbol
            , q.exchange
            , q.event_tm
            , q.event_seq_nb
            , q.bid_pr
            , q.bid_size
            , q.ask_pr
            , q.ask_size
            , t.trade_pr
            , t.mov_avg_trade_pr
            , ROW_NUMBER() OVER (PARTITION BY q.symbol, q.exchange, q.event_tm ORDER BY t.event_tm DESC) as rn
        FROM quotes_curr q
        LEFT JOIN trades_curr_mv_avg t
            ON q.symbol = t.symbol
            AND q.exchange = t.exchange
            AND t.event_tm < q.event_tm
""")
joined_df.createOrReplaceTempView("joined")
joined_df.show(10)

+--------+----------+------+--------+--------------------+------------+------------------+--------+------------------+--------+------------------+--------------------+---+
|rec_type|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|            bid_pr|bid_size|            ask_pr|ask_size|          trade_pr|    mov_avg_trade_pr| rn|
+--------+----------+------+--------+--------------------+------------+------------------+--------+------------------+--------+------------------+--------------------+---+
|       Q|2020-08-06|  SYMC|    NYSE|2020-08-06 09:37:...|           1|160.92615163863337|     100|161.74260959804047|     100|              null|                null|  1|
|       Q|2020-08-06|  SYMC|    NYSE|2020-08-06 09:45:...|           2|161.37519866116340|     100|163.00177900709700|     100|              null|                null|  1|
|       Q|2020-08-06|  SYMC|    NYSE|2020-08-06 09:51:...|           3|159.36830980740774|     100|159.52208902072380|     100|             

In [86]:
final_df = spark.sql("""
    SELECT
        j.trade_dt
        , j.symbol
        , j.exchange
        , j.event_tm
        , j.event_seq_nb
        , j.bid_pr
        , j.bid_size
        , j.ask_pr
        , j.ask_size
        , j.trade_pr AS last_trade_pr
        , j.mov_avg_trade_pr AS last_mov_avg_pr
        , j.bid_pr - c.close_pr AS bid_pr_mv
        , j.ask_pr - c.close_pr AS ask_pr_mv
    FROM joined j
    LEFT JOIN close_pr c
        ON j.symbol = c.symbol
        AND j.exchange = c.exchange
    WHERE j.rn = 1
        AND j.rec_type = 'Q'
""")
final_df.count()

540

In [96]:
final_df.rdd.getNumPartitions()

200

In [ ]:
final_df.coalesce(4).write.partitionBy("trade_dt").mode("overwrite").parquet("ETL-output")
